Содержание:

1. [Задания SQL_1_a](#section1)
2. [Задания SQL_1_b](#section2)
3. [Задания SQL_2](#section3)

In [1]:
import pandas as pd
import psycopg2
from pathlib import Path
import sys
import pprint
import pandasql

In [2]:
conn_string = "host='db' dbname='postgres' user='postgres'  password='example'" # port=5433
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()


In [3]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://postgres:example@db:5432/postgres')

In [4]:
data_path = Path('./data')

Для наглядности создадим таблицы с dummy значениями:

In [5]:
def run_sql_query(query: str, query_name: str, save_result_to_db=False):
    query_result = pandasql.sqldf(query, globals())
    if save_result_to_db:
        query_result.to_sql(query_name + 'result', engine)
    return query_result

<a id='section1'></a>
### **Задание 1.a**

##### Необходимо вывести долю заказов с промокодами

In [6]:
orders = pd.read_csv(data_path / 'task1/orders.csv')
orders


,order_id,promocode_id
0,1,5.0
1,2,5.0
2,3,1.0
3,4,NaN
4,5,2.0
5,6,NaN
6,7,3.0
7,8,3.0
8,9,5.0
9,10,1.0


In [7]:

promocodes = pd.read_csv(data_path / 'task1/promocodes.csv')
promocodes

,promocode_id,name,discount
0,1,name1,10
1,2,name2,5
2,3,name3,5
3,4,name4,7
4,5,name5,3
5,6,name6,5


Количество заказов со скидками:

In [8]:
q1_1 = """
     SELECT COUNT(DISTINCT(orders.order_id))
     FROM orders
     JOIN promocodes ON orders.promocode_id == promocodes.promocode_id;
     """

run_sql_query(q1_1, 'q1_1')

,COUNT(DISTINCT(orders.order_id))
0,8


Общее количество заказов:

In [9]:
q1_1_part2 = """
     SELECT COUNT(*)
     FROM orders;
     """

run_sql_query(q1_1_part2, 'q1_1_part2')

,COUNT(*)
0,10


**Ответ:**
Таким образом, отношение заказов со скидками ко всем заказам равняется 0.8

<a id='section2'></a>
### **Задание 1.b**

##### Необходимо вывести самый популярный промокод (название) и число его использований

In [10]:
q1_2 = """
    SELECT orders.promocode_id, promocodes.name, COUNT(orders.promocode_id) 
    AS value_occurrence 
    FROM orders
    JOIN promocodes ON orders.promocode_id == promocodes.promocode_id 
    GROUP BY orders.promocode_id
    ORDER BY value_occurrence DESC LIMIT 1;
     """

run_sql_query(q1_2, 'q1_2')

,promocode_id,name,value_occurrence
0,5.0,name5,3


<a id='section3'></a>
### **Задание 2**

##### Необходимо вывести топ-10 офисов по потреблению печенек среди офисов, которые потребляют менее 1000 печенек

In [11]:
consumption = pd.read_csv(data_path / 'task2/consumption.csv')
consumption

,coffee_point_id,cookies
0,1,100
1,2,500
2,3,250
3,4,110
4,5,100
5,6,105
6,7,236
7,8,505
8,9,999
9,10,280


In [12]:
offices = pd.read_csv(data_path / 'task2/buildings.csv')
offices

,coffee_point_id,coffee_point_name,office_id,office_name
0,1,name1,1,office1
1,2,name2,1,office1
2,3,name3,1,office1
3,4,name4,2,office2
4,5,name5,2,office2
5,6,name6,3,office3
6,7,name7,4,office4
7,8,name8,5,office5
8,9,name9,5,office5
9,10,name10,6,office6


In [30]:
q2 = """
     SELECT offices.office_id, offices.office_name, sum(consumption.cookies) as num_cookies, 
     offices.coffee_point_id, offices.coffee_point_name
     FROM consumption
     JOIN offices ON consumption.coffee_point_id == offices.coffee_point_id
     GROUP BY offices.office_id
     HAVING num_cookies < 1000
     ORDER BY num_cookies DESC LIMIT 10;
     """

run_sql_query(q2, 'q2')

,office_id,office_name,num_cookies,coffee_point_id,coffee_point_name
0,1,office1,850,1,name1
1,6,office6,280,10,name10
2,4,office4,236,7,name7
3,2,office2,210,4,name4
4,8,office8,185,12,name12
5,3,office3,105,6,name6
6,7,office7,100,11,name11
